* O objetivo deste estudo é a utilização dos algoritmos Decision Tree e Naive Bayes do scikit-learn para prever/inferir a renda baseada nos demais atributos.

In [1]:
# importando bibliotecas
import pandas as pd
import numpy as np
import sklearn.preprocessing

In [2]:
# importação e leitura do dataset
dados = pd.read_csv('census.csv')
dados.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
# verificação do tamanho da base estudada
dados.shape

(32561, 15)

In [4]:
dados.income.value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

In [5]:
# analise geral (dados númericos)
dados.describe()

,age,final-weight,education-num,capital-gain,capital-loos,hour-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


## Pré-Processamento dos Dados

In [6]:
# atributos independentes

# aqui iremos selecionar todas as linhas/ e todas as colunas exceto a ultima (income), pois esta será a nossa coluna de previsão
atributos_previsores = dados.iloc[:,0:14].values
atributos_previsores

array([[39, ' State-gov', 77516, ..., 0, 40, ' United-States'],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 13, ' United-States'],
       [38, ' Private', 215646, ..., 0, 40, ' United-States'],
       ...,
       [58, ' Private', 151910, ..., 0, 40, ' United-States'],
       [22, ' Private', 201490, ..., 0, 20, ' United-States'],
       [52, ' Self-emp-inc', 287927, ..., 0, 40, ' United-States']],
      dtype=object)

In [7]:
# atributo dependente
#aqui iremos selecionar a variável alvo (income), conforme determinado no objetivo do projeto, e todas as linhas do dataset

classe_aprever= dados.iloc[:,14]

- Muito dos nossos atributos são do tipo string, ou seja, são variáveis categóricas. Dessa forma iremos aplicar uma codificação de atributos categóricos para transforma-los em códigos (dados númericos). Caso essa transformação não ocorra não é possivel aplicar os algoritmos de previsão.
Para essa transformação iremos utilizar o LabelEncoder do sklearn. 

Os atributos que precisarão ser preprocessados são: workclass[coluna 1], education [coluna 3], marital-status [coluna 5], occupation [coluna 6], relationship [coluna 7], race [coluna 8], sex [coluna 9] e native-country [coluna 13].

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder_previsores = LabelEncoder()
atributos_previsores[:,1] = label_encoder_previsores.fit_transform(atributos_previsores[:,1])
atributos_previsores[:,3] = label_encoder_previsores.fit_transform(atributos_previsores[:,3])
atributos_previsores[:,5] = label_encoder_previsores.fit_transform(atributos_previsores[:,5])
atributos_previsores[:,6] = label_encoder_previsores.fit_transform(atributos_previsores[:,6])
atributos_previsores[:,7] = label_encoder_previsores.fit_transform(atributos_previsores[:,7])
atributos_previsores[:,8] = label_encoder_previsores.fit_transform(atributos_previsores[:,8])
atributos_previsores[:,9] = label_encoder_previsores.fit_transform(atributos_previsores[:,9])
atributos_previsores[:,13] = label_encoder_previsores.fit_transform(atributos_previsores[:,13])
atributos_previsores

array([[39, 7, 77516, ..., 0, 40, 39],
       [50, 6, 83311, ..., 0, 13, 39],
       [38, 4, 215646, ..., 0, 40, 39],
       ...,
       [58, 4, 151910, ..., 0, 40, 39],
       [22, 4, 201490, ..., 0, 20, 39],
       [52, 5, 287927, ..., 0, 40, 39]], dtype=object)

* Depois de transformar as variáveis categóricas em númericos, é necessário aplicar binarização dos valores inteiros. Esse processo é extremamente importante, pois não permite que o algoritmo considere um valor maior mais importante que um valor menor. Dessa forma, colocamos todos em uma escala de 0 a 1. 

Um método muito utilizado e eficiente é o OneHotEnconder, e é este que iremos utilizar nesta etapa do pré-processamento.

In [9]:
from sklearn.preprocessing import OneHotEncoder

OHE = OneHotEncoder(categories='auto')
atributos_previsores = OHE.fit_transform(atributos_previsores).toarray()
atributos_previsores

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

* Para aplicar um algoritmo de Machine Learning, seja ele qual for, é sempre necessário dividir os dados em dois conjuntos: treino e teste. O treino deve ser o maior conjunto, e será utilizado literalmente para treinar o nosso modelo. E o conjunto de teste será os dados que o modelo utilizará para testar as suas saidas. 
Para esse projeto iremos utilizar 25% da base para teste e os demais para treino. Como ferramenta de divisão iremos utilizar o train_test_split do sklearn. 

In [10]:
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(atributos_previsores, classe_aprever , test_size = 0.25, random_state = 0)

## Criando os modelos de previsão

```
```



## Primeiro Modelo - Decision Tree Classifier

* Parametros
  - Iremos utilizar a função entropia para medir a qualidade da seleção dos nós


In [11]:
from sklearn.tree import DecisionTreeClassifier

classificadorDTC = DecisionTreeClassifier(criterion='entropy', random_state=0)
classificadorDTC.fit(previsores_treinamento, classe_treinamento)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [12]:
# Aplicação de teste do modelo
classe_prevista_teste_DTC = classificadorDTC.predict(previsores_teste)

## Segundo Modelo - Naive Bayes




In [13]:
from sklearn.naive_bayes import GaussianNB

classificadorNB = GaussianNB()
classificadorNB.fit(previsores_treinamento, classe_treinamento)

GaussianNB()

In [14]:
# Aplicação de teste do modelo
classe_prevista_teste_NB = classificadorNB.predict(previsores_teste)

## Performance dos modelos

- Após aplicar os modelos no nosso dataset é muito impportante aplicar técnicas de verificação, pois isso permite que possamos compreender se o resultado que estamos obtendo é satisfatório, ou se cabe alguma melhoria.

## Performance - Decision Tree Classifier

* Iremos calular a acurácia do modelo como técnica de análise através do accuracy_score do sklearn

In [15]:
from sklearn.metrics import accuracy_score

precisao_modelo_DTC = accuracy_score(classe_teste, classe_prevista_teste_DTC)
print('Acurácia do Modelo: %.2f%% ' % (precisao_modelo_DTC*100))

Acurácia do Modelo: 84.61% 


* Análise: 84.61% de acurácia é um ótimo resultado.

## Performance - Naive Bayes

* Iremos calular a acurácia do modelo como técnica de análise através do accuracy_score do sklearn

In [16]:
precisao_modelo_NB = accuracy_score(classe_teste, classe_prevista_teste_NB)
print('Acurácia do Modelo: %.2f%% ' % (precisao_modelo_NB *100))

Acurácia do Modelo: 50.41% 


* Análise: 50.41% de acurácia é um MAL/PÉSSIMO resultado. E possivelmente, isso se deve a forma como utilizamos como pré-processamos os dados. Pesquisando sobre o algoritmo Naive Bayes (na documentação do metodo e biblioteca sklearn), concluimos que o pré-processamento não contribui com a eficiência do algoritmo, o que podemos provar através dessa acurácia.
    Dessa forma vamos aplicar outro método de pré-processamento conforme pesquisa, o LabelEnconder nos atributos categóricos, mas iremos aplicar o método StandardScaler nas colunas que já eram númericas. O objetivo é colocar todos os valores na mesma escala.

# A partir daqui iremos realizar toda a etapa de pré-processamento novamente

In [17]:
# atributos independentes

# aqui iremos selecionar todas as linhas/ e todas as colunas exceto a ultima (income), pois esta será a nossa coluna de previsão
atributos_previsores = dados.iloc[:,0:14].values
atributos_previsores

array([[39, ' State-gov', 77516, ..., 0, 40, ' United-States'],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 13, ' United-States'],
       [38, ' Private', 215646, ..., 0, 40, ' United-States'],
       ...,
       [58, ' Private', 151910, ..., 0, 40, ' United-States'],
       [22, ' Private', 201490, ..., 0, 20, ' United-States'],
       [52, ' Self-emp-inc', 287927, ..., 0, 40, ' United-States']],
      dtype=object)

In [18]:
# atributo dependente
#aqui iremos selecionar a variável alvo (income), conforme determinado no objetivo do projeto, e todas as linhas do dataset

classe_aprever= dados.iloc[:,14]

In [19]:
label_encoder_previsores = LabelEncoder()
atributos_previsores[:,1] = label_encoder_previsores.fit_transform(atributos_previsores[:,1])
atributos_previsores[:,3] = label_encoder_previsores.fit_transform(atributos_previsores[:,3])
atributos_previsores[:,5] = label_encoder_previsores.fit_transform(atributos_previsores[:,5])
atributos_previsores[:,6] = label_encoder_previsores.fit_transform(atributos_previsores[:,6])
atributos_previsores[:,7] = label_encoder_previsores.fit_transform(atributos_previsores[:,7])
atributos_previsores[:,8] = label_encoder_previsores.fit_transform(atributos_previsores[:,8])
atributos_previsores[:,9] = label_encoder_previsores.fit_transform(atributos_previsores[:,9])
atributos_previsores[:,13] = label_encoder_previsores.fit_transform(atributos_previsores[:,13])
atributos_previsores

array([[39, 7, 77516, ..., 0, 40, 39],
       [50, 6, 83311, ..., 0, 13, 39],
       [38, 4, 215646, ..., 0, 40, 39],
       ...,
       [58, 4, 151910, ..., 0, 40, 39],
       [22, 4, 201490, ..., 0, 20, 39],
       [52, 5, 287927, ..., 0, 40, 39]], dtype=object)

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

atributos_previsores[:,0] =  scaler.fit_transform(atributos_previsores[:,0].reshape(-1,1)).ravel()
atributos_previsores[:,2] =  scaler.fit_transform(atributos_previsores[:,2].reshape(-1,1)).ravel()
atributos_previsores[:,4] =  scaler.fit_transform(atributos_previsores[:,4].reshape(-1,1)).ravel()
atributos_previsores[:,10] =  scaler.fit_transform(atributos_previsores[:,10].reshape(-1,1)).ravel()
atributos_previsores[:,11] =  scaler.fit_transform(atributos_previsores[:,11].reshape(-1,1)).ravel()
atributos_previsores[:,12] =  scaler.fit_transform(atributos_previsores[:,12].reshape(-1,1)).ravel()

In [21]:
# aplicar novamente o train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(atributos_previsores, classe_aprever , test_size = 0.25, random_state = 0)

In [22]:
#aplica o modelo novamente
classificadorNB = GaussianNB()
classificadorNB.fit(previsores_treinamento, classe_treinamento)

GaussianNB()

In [23]:
#testa novamente
classe_prevista_teste_NB = classificadorNB.predict(previsores_teste)

In [24]:
# calcula a acurácia novamente
precisao_modelo_NB = accuracy_score(classe_teste, classe_prevista_teste_NB)
print('Acurácia do Modelo: %.2f%% ' % (precisao_modelo_NB *100))

Acurácia do Modelo: 80.46% 


* Análise: Pronto, agora obtemos um resultado muito melhor, próximo ao alcançando no Decision Tree Classifier, no entanto ainda inferior ao outro método.

## Conclusão

# Decision Tree Classifier

In [25]:
# Comparando a Classe Prevista com a Classe Real

df = pd.DataFrame( )
df['Classe_Prevista_DTC'] = classe_prevista_teste_DTC
df['Classe_Real'] = classe_teste.values
df['Comparacao'] = df['Classe_Prevista_DTC'] == df['Classe_Real']

df_comparacao = df.groupby(by='Comparacao').count()

print('De %d classes o modelo DECISION TREE CLASSIFIER acertou %d classes e errou %d classes.\n ' % (df_comparacao['Classe_Prevista_DTC'].sum(), df_comparacao.iloc[[1], [0]].values, df_comparacao.iloc[[0], [0]].values))

df_comparacao

De 8141 classes o modelo DECISION TREE CLASSIFIER acertou 6888 classes e errou 1253 classes.
 


,Classe_Prevista_DTC,Classe_Real
Comparacao,,
False,1253,1253
True,6888,6888


## Naive Bayes

In [26]:
# Comparando a Classe Prevista com a Classe Real

df = pd.DataFrame( )
df['Classe_Prevista_NB'] = classe_prevista_teste_NB
df['Classe_Real'] = classe_teste.values
df['Comparacao'] = df['Classe_Prevista_NB'] == df['Classe_Real']

df_comparacao = df.groupby(by='Comparacao').count()

print('De %d classes o modelo NAIVE BAYES acertou %d classes e errou %d classes.\n ' % (df_comparacao['Classe_Prevista_NB'].sum(), df_comparacao.iloc[[1], [0]].values, df_comparacao.iloc[[0], [0]].values))

df_comparacao

De 8141 classes o modelo NAIVE BAYES acertou 6550 classes e errou 1591 classes.
 


,Classe_Prevista_NB,Classe_Real
Comparacao,,
False,1591,1591
True,6550,6550


* Concluimos que, de acordo com o relatório aqui proposto podemos observar nos resultados acima que o 'Algoritmo Decision Tree' obteve um melhor resultado em relação ao 'Algoritmo Naive Bayes' para o dataset 'census.csv'.

O Algoritmo Decision Tree Classifier foi superior ao Algoritmo Naive Bayes em relação a acurácia e ao teste de comparação de acertos de classe.

Ainda reitero a importancia da utilização de técnicas de validação, como por exemplo o score da acurácia que nos permitiu realizar o pré-processamento dos dados novamente para aplicação no algoritmo Naive Bayes. Esse tipo de trabalho contribui para que possamos estar em contato com situações reais e buscar informações para resolver as questões da maneira mais efetiva possivel. 